# Test stuff and explore FCN with VGG16 example

In [40]:
import os.path
import tensorflow as tf
import helper
import cv2
import numpy as np
import scipy.misc
import warnings
from distutils.version import LooseVersion
import project_tests as tests

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 75%; }
    div#menubar-container     { width: 75%; }
    div#maintoolbar-container { width: 75%; }
</style>"""))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


data_dir = './data'
runs_dir = './runs'

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

Default GPU Device: /device:GPU:0


In [4]:
# Path to vgg model
vgg_tag = 'vgg16'
vgg_path = os.path.join(data_dir, 'vgg')

def getGraph(sess):
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    return graph

## Print all variables / layer names in graph

In [13]:
tf.reset_default_graph()
with tf.Session() as sess:
    graph = getGraph(sess)
    saver = tf.train.Saver()
    saver.restore(sess, "./runs/semantic_segmentation_model.ckpt")
    for i in graph.get_operations():
        print("{}\n\t{}".format(i.name, i.values()))

INFO:tensorflow:Restoring parameters from ./data/vgg/variables/variables
INFO:tensorflow:Restoring parameters from ./runs/semantic_segmentation_model.ckpt
image_input
	(<tf.Tensor 'image_input:0' shape=(?, ?, ?, 3) dtype=float32>,)
keep_prob
	(<tf.Tensor 'keep_prob:0' shape=<unknown> dtype=float32>,)
Processing/split/split_dim
	(<tf.Tensor 'Processing/split/split_dim:0' shape=() dtype=int32>,)
Processing/split
	(<tf.Tensor 'Processing/split:0' shape=(?, ?, ?, 1) dtype=float32>, <tf.Tensor 'Processing/split:1' shape=(?, ?, ?, 1) dtype=float32>, <tf.Tensor 'Processing/split:2' shape=(?, ?, ?, 1) dtype=float32>)
Processing/sub/y
	(<tf.Tensor 'Processing/sub/y:0' shape=() dtype=float32>,)
Processing/sub
	(<tf.Tensor 'Processing/sub:0' shape=(?, ?, ?, 1) dtype=float32>,)
Processing/sub_1/y
	(<tf.Tensor 'Processing/sub_1/y:0' shape=() dtype=float32>,)
Processing/sub_1
	(<tf.Tensor 'Processing/sub_1:0' shape=(?, ?, ?, 1) dtype=float32>,)
Processing/sub_2/y
	(<tf.Tensor 'Processing/sub_2/y:0' 

## Get trainable variables

In [14]:
for var in tf.trainable_variables():
    print(var)

<tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_2/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_2/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_3/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_3/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv4_1/filter:0' shape=(3, 

## Get global variables

In [15]:
for var in tf.global_variables():
    print(var)

<tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_2/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_2/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_3/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_3/biases:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv4_1/filter:0' shape=(3, 

## Test inference on video
Necessary imports and initialization:

In [65]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from moviepy.editor import *

import helper
import project_tests as tests
from main import load_vgg, layers, optimize, train_nn, test_nn

tf.reset_default_graph()

model_checkpoint = "./runs/semantic_segmentation_model.ckpt"
num_classes = 2

image_shape = (160, 576)

video_fps = 30
video_output_folder = "videos_output/"
videos = [
   "data/project_video.mp4",
   "data/challenge_video.mp4",
   "data/harder_challenge_video.mp4"
]

def process_video_image(sess, logits, keep_prob, image_input_op, image_src, image_shape):
    # first crop image to correct aspect of `image_shape`
    image_src_shape  = image_src.shape
    new_y = (image_shape[0] * image_src_shape[1]) // image_shape[1]
    image_crop = image_src[new_y:,:]
    image_resized = scipy.misc.imresize(image_crop, image_shape)

    feed_dict = {keep_prob: 1.0,
                 image_input_op: [image_resized]}
    im_softmax = sess.run([tf.nn.softmax(logits)],
                          feed_dict=feed_dict)
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    street_im = scipy.misc.toimage(image_resized)
    street_im.paste(mask, box=None, mask=mask)
    return np.asarray(street_im)

In [69]:
video_fps = 10
clip_part = (0.0, 6.0)

tf.reset_default_graph()
# TF placeholders
correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name="correct_label")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

with tf.Session() as sess:
    # Path to vgg model
    vgg_path = os.path.join(data_dir, "vgg")

    # Build NN using load_vgg, layers
    image_input, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
    last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

    tvars = tf.trainable_variables()
    trainable_vars = [var for var in tvars if "fc6" not in var.name and 
                                              "fc7" not in var.name]

    # Set-up optimizer
    return_list = optimize(last_layer, correct_label, learning_rate, num_classes, trainable_vars)
    logits_op, train_op, loss_op, mean_iou_value, mean_iou_update_op = return_list

    #graph = tf.get_default_graph()
    saver = tf.train.Saver()
    try:
        saver.restore(sess, model_checkpoint)
    except:
        print("Couldn't load model last checkpoint ({}).".format(model_checkpoint))
        print("You need to either provide the required checkpoint files or train the network from scratch!")
  
    input_image_op  = graph.get_tensor_by_name("image_input:0")
    logits_op       = graph.get_tensor_by_name("decoder_logits:0")
    keep_prob       = graph.get_tensor_by_name("keep_prob:0")
    
    for video in videos:
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)
        result_path = video_output_folder + os.path.basename(video)
        if not os.path.isfile(video):
            print("Video {} doesn't exist!".format(video))
        else:
            clip1 = VideoFileClip(video) #.subclip(*clip_part)
            video_slowdown_factor = video_fps / clip1.fps
            clip1 = clip1.fx(vfx.speedx, video_slowdown_factor)
            white_clip = clip1.fl_image(lambda img: process_video_image(sess, logits_op, keep_prob, input_image_op, img, image_shape))
            %time white_clip.write_videofile(result_path, audio=False, fps=video_fps)

INFO:tensorflow:Restoring parameters from ./data/vgg/variables/variables


KeyboardInterrupt: 

In [53]:
HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format("videos_output/harder_challenge_video.mp4"))

In [72]:
tf.reset_default_graph()

# TF placeholders
correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name="correct_label")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

# Create a builder
builder = tf.saved_model.builder.SavedModelBuilder('./saved/')

with tf.Session() as sess:
    # Path to vgg model
    vgg_path = os.path.join(data_dir, "vgg")

    # Build NN using load_vgg, layers
    image_input, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
    last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

    tvars = tf.trainable_variables()
    trainable_vars = [var for var in tvars if "fc6" not in var.name and 
                                              "fc7" not in var.name]

    # Set-up optimizer
    return_list = optimize(last_layer, correct_label, learning_rate, num_classes, trainable_vars)
    logits_op, train_op, loss_op, mean_iou_value, mean_iou_update_op = return_list

    #graph = tf.get_default_graph()
    saver = tf.train.Saver()
    try:
        saver.restore(sess, model_checkpoint)
    except:
        print("Couldn't load model last checkpoint ({}).".format(model_checkpoint))
        print("You need to either provide the required checkpoint files or train the network from scratch!")
  

    builder.add_meta_graph_and_variables(sess,
                                       [tf.saved_model.tag_constants.TRAINING],
                                       signature_def_map=None,
                                       assets_collection=None)
builder.save()  

INFO:tensorflow:Restoring parameters from ./data/vgg/variables/variables
INFO:tensorflow:Restoring parameters from ./runs/semantic_segmentation_model.ckpt
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved/saved_model.pb


b'./saved/saved_model.pb'